<a href="https://colab.research.google.com/github/azml-154/CNN-1/blob/main/bestMNISTmodel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#import window
from keras import models, layers
from keras import Input
from keras.models import Sequential, load_model, Model
from keras import initializers, regularizers, metrics
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.layers import BatchNormalization, Conv2D, Activation, Dense, GlobalAveragePooling2D, MaxPooling2D, ZeroPadding2D, Add, Flatten, Dropout
from keras.models import load_model
from keras.datasets import mnist
from keras.utils import np_utils
from time import time
from keras.optimizers import SGD
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

import cv2
import os
import matplotlib.pyplot as plt
import numpy as np
import math
import tensorflow as tf

In [ ]:
MY_EPOCH = 10000
MY_BATCH = 256
num_classes = 10

In [ ]:
batch_size = 128
num_classes = 10
epochs = 12

(X_train, Y_train), (X_test, Y_test) = mnist.load_data()

input_shape = (28, 28, 1)
print(X_train.shape)
print(X_test.shape)


#이진화 적용 (오츄 알고리즘)
for i in range(60000):
    _, X_train[i] = cv2.threshold(X_train[i], 0, 255, cv2.THRESH_OTSU)

for i in range(10000):
    _, X_test[i] = cv2.threshold(X_test[i], 0, 255, cv2.THRESH_OTSU)



#rescale
X_train = X_train.astype('float32') / 255.
X_test = X_test.astype('float32') / 255.

#input shape
X_train = X_train.reshape(X_train.shape[0], 28, 28, 1)
X_test = X_test.reshape(X_test.shape[0], 28, 28, 1)

print('x_train shape:', X_train.shape)


#one hot encoding
Y_train = np_utils.to_categorical(Y_train, num_classes)
Y_test = np_utils.to_categorical(Y_test, num_classes)

11493376/11490434 [==============================] - 0s 0us/step
(60000, 28, 28)
(10000, 28, 28)
x_train shape: (60000, 28, 28, 1)


In [ ]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), strides=(1, 1), padding='same',
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))

model.add(MaxPooling2D(pool_size=(2,2), strides=(1, 1)))

model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())


model.add(Dense(num_classes, activation='softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 28, 28, 32)        320       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 28, 28, 64)        18496     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 27, 27, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 27, 27, 128)       73856     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 27, 27, 128)       147584    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 128)       0         
_________________________________________________________________
flatten (Flatten)            (None, 21632)             0

In [ ]:
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics= ['acc'])

model_DIR='./model/'
if not os.path.exists(model_DIR):
    os.mkdir(model_DIR)

path='./model/{epoch:02d}-{val_loss:.4f}.h5'
checkpointer = ModelCheckpoint(filepath=path, monitor='val_loss', verbose=1, save_best_only=True)
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=30)


# 새 섹션

In [ ]:
print('train start')
begin = time()
model.fit(X_train, Y_train, epochs = MY_EPOCH, batch_size= MY_BATCH, verbose = 1, validation_data=(X_test,Y_test), callbacks=[early_stopping_callback, checkpointer])
end = time()
print("train time : ", end-begin)

train start
Epoch 1/10000
235/235 [==============================] - 49s 66ms/step - loss: 0.4580 - acc: 0.8524 - val_loss: 0.0525 - val_acc: 0.9836

Epoch 00001: val_loss improved from inf to 0.05246, saving model to ./model/01-0.0525.h5
Epoch 2/10000
235/235 [==============================] - 15s 62ms/step - loss: 0.0552 - acc: 0.9830 - val_loss: 0.0497 - val_acc: 0.9839

Epoch 00002: val_loss improved from 0.05246 to 0.04968, saving model to ./model/02-0.0497.h5
Epoch 3/10000
235/235 [==============================] - 15s 63ms/step - loss: 0.0338 - acc: 0.9893 - val_loss: 0.0412 - val_acc: 0.9870

Epoch 00003: val_loss improved from 0.04968 to 0.04124, saving model to ./model/03-0.0412.h5
Epoch 4/10000
235/235 [==============================] - 15s 64ms/step - loss: 0.0283 - acc: 0.9910 - val_loss: 0.0348 - val_acc: 0.9882

Epoch 00004: val_loss improved from 0.04124 to 0.03477, saving model to ./model/04-0.0348.h5
Epoch 5/10000
235/235 [==============================] - 15s 64ms/st